In [9]:
import polars as pl
df = pl.scan_parquet('/dev/shm/013000_013500.pq').filter(pl.col.id < 13100).collect()
df

datetime,close,id
datetime[ms],i32,i16
2010-07-05 09:23:02,0,13000
2010-07-05 09:23:07,0,13000
2010-07-05 09:23:12,0,13000
2010-07-05 09:23:17,0,13000
2010-07-05 09:23:22,0,13000
…,…,…
2024-12-31 14:56:51.290,426,13099
2024-12-31 14:56:54.289,427,13099
2024-12-31 14:56:57.377,426,13099


In [17]:
df = df.filter(pl.col.close > 0)

In [20]:
df['close']

close
i32
800
800
799
799
793
…
426
427
426


In [10]:
df['id'].unique()

id
i16
13000
13001
13002
13003
13005
…
13088
13089
13096


In [21]:
df.group_by('id').agg(pl.col.datetime.last() - pl.col.datetime.first())

id,datetime
i16,duration[ms]
13002,5475d 5h 37m 793ms
13005,5475d 5h 37m 709ms
13008,5475d 5h 36m 56s 151ms
13011,5046d 5h 34m 57s 585ms
13020,3212d 5h 34m 58s 91ms
…,…
13022,754d 5h 37m 7s 286ms
13028,4803d 5h 34m 58s 770ms
13058,4933d 5h 35m 1s 304ms


In [23]:
df.filter(pl.col.id == 13002)

datetime,close,id
datetime[ms],i32,i16
2010-01-04 09:23:04,793,13002
2010-01-04 09:23:24,793,13002
2010-01-04 09:23:49,793,13002
2010-01-04 09:23:54,793,13002
2010-01-04 09:24:04,793,13002
…,…,…
2024-12-31 14:56:52.794,434,13002
2024-12-31 14:56:55.811,434,13002
2024-12-31 14:56:58.804,435,13002


In [36]:
x = df.filter(pl.col.datetime.dt.time() < time(9, 25, 00)).group_by('id', pl.col.datetime.dt.date()).agg(pl.col.close.unique())


In [35]:
df = df.filter(pl.col.datetime.dt.time() >= time(9, 24, 50))

In [39]:
x.sort(pl.col.close.list.len())

id,datetime,close
i16,date,list[i32]
13020,2020-04-16,[840]
13088,2012-04-24,[2640]
13015,2022-10-11,[504]
13099,2022-09-22,[265]
13005,2021-06-09,[270]
…,…,…
13099,2014-03-31,"[632, 634]"
13006,2011-10-21,"[734, 738]"
13002,2011-10-21,"[1284, 1286]"


In [44]:
from datetime import time, date
df.filter(pl.col.datetime.dt.time() <= time(9, 30, 20)).filter(pl.col.id == 13002).filter(pl.col.datetime.dt.date() == date(2011, 10, 21))

datetime,close,id
datetime[ms],i32,i16
2011-10-21 09:24:59,1286,13002
2011-10-21 09:24:59,1284,13002
2011-10-21 09:30:05,1280,13002
2011-10-21 09:30:09,1280,13002
2011-10-21 09:30:14,1280,13002
2011-10-21 09:30:19,1280,13002


In [46]:
y = df.select((pl.col.datetime - pl.col.datetime.shift(1).over('id')))

In [47]:
y.drop_nulls()

datetime
duration[ms]
4m 55s
5s
5s
5s
5s
…
3s 1ms
2s 999ms
3s 88ms


In [52]:
y.filter(pl.col.datetime < pl.duration(seconds=2))


datetime
duration[ms]
0ms
0ms
0ms
0ms
0ms
…
83ms
2ms
82ms


In [56]:
df.filter(pl.col.datetime == pl.col.datetime.shift(1).over('id'))

datetime,close,id
datetime[ms],i32,i16
2010-07-05 09:31:37,797,13000
2010-07-05 11:08:28,795,13000
2010-07-05 13:01:38,795,13000
2010-07-05 14:53:33,789,13000
2010-07-07 10:38:51,764,13000
…,…,…
2019-04-12 13:26:50,424,13099
2019-04-12 13:39:47,421,13099
2019-04-12 13:54:23,422,13099


In [57]:
df.filter(pl.col.datetime == pl.col.datetime.shift(-1).over('id'))

datetime,close,id
datetime[ms],i32,i16
2010-07-05 09:31:37,794,13000
2010-07-05 11:08:28,796,13000
2010-07-05 13:01:38,796,13000
2010-07-05 14:53:33,788,13000
2010-07-07 10:38:51,765,13000
…,…,…
2019-04-12 13:26:50,423,13099
2019-04-12 13:39:47,422,13099
2019-04-12 13:54:23,421,13099


In [55]:
df = df.filter(pl.col.datetime.ne(pl.col.datetime.shift(1).over('id')) | pl.col.close.ne(pl.col.close.shift(1).over('id')))

In [58]:
sorry = df.filter(pl.col.datetime == pl.col.datetime.shift(1).over('id'))

In [ ]:
df = df.filter(pl.col.datetime != pl.col.datetime.shift(1).over('id'))

In [ ]:
delta = df.with_columns(delta = pl.when(pl.col.id.eq(pl.col.id.shift(1) & pl.col.datetime-pl.col.datetime.shift(1)<pl.duration(hours=1))).then(pl.col.close - pl.col.close.shift(1)).otherwise(-9999)
                  ).select(pl.when(pl.col.delta.abs() < 64).then(pl.col.delta + 64).otherwise(0).cast(pl.UInt8))

datetime,close,id
datetime[ms],i32,i16
2010-07-05 09:30:07,799,13000
2010-07-05 09:30:12,799,13000
2010-07-05 09:30:17,793,13000
2010-07-05 09:30:22,798,13000
2010-07-05 09:30:27,794,13000
…,…,…
2024-12-31 14:56:51.290,426,13099
2024-12-31 14:56:54.289,427,13099
2024-12-31 14:56:57.377,426,13099


In [65]:
x = df.select((pl.col.datetime.dt.round('1s') - pl.col.datetime.dt.round('1s').shift(1).over('id')).value_counts())
pl.Config.set_tbl_rows(199)

x

datetime
struct[2]
"{25s,323902}"
"{9d 18h 27m 58s,2}"
"{22s,31914}"
"{2d 18h 24m 37s,22}"
"{1h 29m 43s,2}"
"{2d 19h 30m 1s,1}"
"{1h 30m 49s,12}"
"{1h 34m 30s,1}"
"{1h 31m 18s,3}"


In [66]:
x.sort('datetime')

datetime
struct[2]
"{null,37}"
"{0ms,14674}"
"{1s,4019448}"
"{2s,15850625}"
"{3s,166547691}"
"{4s,18554633}"
"{5s,44602178}"
"{6s,17586926}"
"{7s,3045286}"
